In [1]:
import asyncio
from playwright.async_api import async_playwright
from time import sleep



In [69]:
def save_names(f_names, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for f_name in f_names:
            f.write(f_name)


# TODO:
# figure out how to get rid of "Verified" when writing to file
async def get_names(page, limit, filename):
    await page.keyboard.press("Tab")
    await page.keyboard.press("Tab")

    f_names = set()

    for i in range(limit):
        for _ in range(50):
            await page.keyboard.press("ArrowDown")

        if i % 10 == 0:
            tmp_name_list = await page.locator("role=link").all_inner_texts()
            f_names.update(tmp_name_list)
            if len(f_names) >= limit:
                break

        sleep(0.75)

    tmp_name_list = await page.locator("role=link").all_inner_texts()
    f_names.update(tmp_name_list)

    for name in ("", "primetimetank_", "explore", "Verified"):
        try:
            f_names.remove(name)
        except:
            continue

    with open(filename, "w", encoding="utf-8") as f:
        for f_name in f_names:
            f.write(f"https://www.instagram.com/{f_name}\n")

    f_names.clear()

    with open(filename, "r", encoding="utf-8") as f:
        for f_name in f.readlines():
            if f_name != "Verified":
                f_names.add(f_name)

    save_names(f_names=f_names, filename=filename)

    return f_names


async def run(playwright):
    firefox = playwright.firefox
    browser = await firefox.launch(headless=False)
    context = await browser.new_context(storage_state="instagram.json")
    page = await context.new_page()
    url = "https://www.instagram.com/primetimetank_"
    await page.goto(url)
    sleep(10)

    followers_amount = await page.locator("text=followers").all_inner_texts()
    followers_amount = int(followers_amount[0].split(" ")[0].replace(",", ""))

    following_amount = await page.locator("text=following").all_inner_texts()
    following_amount = int(following_amount[0].split(" ")[0].replace(",", ""))

    await page.goto(f"{url}/followers")
    sleep(4)

    followers_names = await get_names(
        page=page,
        limit=followers_amount,
        filename="followers_links.txt",
    )
    print(f"Followers: {len(followers_names)}")

    await page.goto(f"{url}/following")
    sleep(4)
    following_names = await get_names(
        page=page,
        limit=following_amount,
        filename="following_links.txt",
    )
    print(f"Following: {len(following_names)}")

    not_following_me_back = following_names.difference(followers_names)
    im_not_following_back = followers_names.difference(following_names)

    print(f"Not following me back: {len(not_following_me_back)}")
    save_names(f_names=not_following_me_back, filename="not_following_me_back.txt")

    print(f"I'm not following back: {len(im_not_following_back)}")
    save_names(f_names=im_not_following_back, filename="im_not_following_back.txt")

    await context.close()
    await browser.close()


async def main():
    async with async_playwright() as playwright:
        await run(playwright)


In [3]:
async with async_playwright() as playwright:
    firefox = playwright.firefox
    browser = await firefox.launch(headless=False)
    context = await browser.new_context(storage_state="instagram.json")
    page = await context.new_page()
    url = "https://www.instagram.com/primetimetank_"
    await page.goto(url)

In [4]:
firefox

<BrowserType name=firefox executable_path=/home/tank/.cache/ms-playwright/firefox-1344/firefox/firefox>

In [82]:
pr = await async_playwright().start()
firefox = pr.firefox
browser = await firefox.launch(headless=False)
context = await browser.new_context(storage_state="instagram.json")
page = await context.new_page()
url = "https://www.instagram.com/primetimetank_"
await page.goto(url)

<Response url='https://www.instagram.com/primetimetank_/' request=<Request url='https://www.instagram.com/primetimetank_/' method='GET'>>

In [83]:
followers_amount = await page.locator("text=followers").all_inner_texts()
followers_amount = int(followers_amount[0].split(" ")[0].replace(",", ""))

following_amount = await page.locator("text=following").all_inner_texts()
following_amount = int(following_amount[0].split(" ")[0].replace(",", ""))

In [84]:
followers_amount, following_amount

(1286, 758)

In [85]:
await page.goto(f"{url}/followers")

<Response url='https://www.instagram.com/primetimetank_/followers/' request=<Request url='https://www.instagram.com/primetimetank_/followers/' method='GET'>>

In [86]:
followers_names = await get_names(
    page=page,
    text="Followers",
    nth=-2,
    limit=followers_amount,
    filename="followers_links.txt",
)
print(f"Followers: {len(followers_names)}")


Followers: 1287


In [91]:
await page.goto(f"{url}/following")

<Response url='https://www.instagram.com/primetimetank_/following/' request=<Request url='https://www.instagram.com/primetimetank_/following/' method='GET'>>

In [92]:
following_names = await get_names(
    page=page,
    text="Following",
    nth=3,
    limit=following_amount,
    filename="following_links.txt",
)
print(f"Following: {len(following_names)}")

Following: 759


In [93]:
not_following_me_back = following_names.difference(followers_names)
im_not_following_back = followers_names.difference(following_names)

print(f"Not following me back: {len(not_following_me_back)}")
save_names(f_names=not_following_me_back, filename="not_following_me_back.txt")

print(f"I'm not following back: {len(im_not_following_back)}")
save_names(f_names=im_not_following_back, filename="im_not_following_back.txt")

Not following me back: 57
I'm not following back: 585


In [94]:
await context.close()
await browser.close()